#### Hemlata Channe
#### NLP Project 1 Part 2
#### DOMAIN: Customer support
• CONTEXT: Great Learning has a an academic support department which receives numerous support requests every day throughout the
year. Teams are spread across geographies and try to provide support round the year. Sometimes there are circumstances where due to
heavy workload certain request resolutions are delayed, impacting company’s business. Some of the requests are very generic where a
proper resolution procedure delivered to the user can solve the problem. Company is looking forward to design an automation which can
interact with the user, understand the problem and display the resolution procedure [ if found as a generic request ] or redirect the request
to an actual human support executive if the request is complex or not in it’s database.
#### DATA DESCRIPTION: A sample corpus is attached for your reference. Please enhance/add more data to the corpus using your linguistics
skills.
#### PROJECT OBJECTIVE: Design a python based interactive semi - rule based chatbot which can do the following:
1. Start chat session with greetings and ask what the user is looking for.
2. Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus.
3. End the chat session only if the user requests to end else ask what the user is looking for. Loop continues till the user asks to end it.
Please use the sample chatbot demo video for reference.
##### EVALUATION: GL evaluator will use linguistics to twist and turn sentences to ask questions on the topics described in DATA DESCRIPTION and check if the bot is giving relevant replies.
Hint: There are a lot of techniques using which one can clean and prepare the data which can be used to train a ML/DL classifier. Hence, it might require you to experiment,research, self learn and implement the above classifier. There might be many iterations between hand building the corpus and designing the best it text classifier. As the quality and quantity of corpus increases the model’s performance i.e. ability to answer right questions also increases.
Reference: https://www.mygreatlearning.com/blog/basics-of-building-an-artificial-intelligence-chatbot/

In [1]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import json 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

2022-01-14 20:49:50.415574: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2022-01-14 20:49:50.415907: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2022-01-14 20:49:50.415946: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) 
#nltk.download('punkt') 
#nltk.download('wordnet') 


True

In [3]:
import json
with open('GL Bot.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)
print(num_classes)

8


In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

2022-01-14 20:50:12.261198: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-01-14 20:50:12.409290: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-01-14 20:50:12.409388: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lenovo-Lenovo-ideapad-520-15IKB): /proc/driver/nvidia/version does not exist
2022-01-14 20:50:12.411885: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-01-14 20:50:12.661795: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 1800000000 Hz
2022-01-14 20:50:12.664364: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562e033f30b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-01-14 20:50

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [7]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Train on 138 samples
Epoch 1/500
138/138 [==============================] - 2s 13ms/sample - loss: 2.0813 - accuracy: 0.0725
Epoch 2/500
138/138 [==============================] - 0s 143us/sample - loss: 2.0755 - accuracy: 0.2391
Epoch 3/500
138/138 [==============================] - 0s 191us/sample - loss: 2.0725 - accuracy: 0.2391
Epoch 4/500
138/138 [==============================] - 0s 218us/sample - loss: 2.0693 - accuracy: 0.2391
Epoch 5/500
138/138 [==============================] - 0s 134us/sample - loss: 2.0658 - accuracy: 0.2391
Epoch 6/500
138/138 [==============================] - 0s 112us/sample - loss: 2.0623 - accuracy: 0.2391
Epoch 7/500
138/138 [==============================] - 0s 175us/sample - loss: 2.0584 - accuracy: 0.2391
Epoch 8/500
138/138 [==============================] - 0s 134us/sample - loss: 2.0543 - accuracy: 0.2391
Epoch 9/500
138/138 [==============================] - 0s 177us/sample - loss: 2.0501 - accuracy: 0.2391
Epoch 10/500
138/138 [=============

138/138 [==============================] - 0s 102us/sample - loss: 1.3361 - accuracy: 0.5870
Epoch 79/500
138/138 [==============================] - 0s 88us/sample - loss: 1.3237 - accuracy: 0.5797
Epoch 80/500
138/138 [==============================] - 0s 84us/sample - loss: 1.3085 - accuracy: 0.5797
Epoch 81/500
138/138 [==============================] - 0s 90us/sample - loss: 1.2953 - accuracy: 0.5797
Epoch 82/500
138/138 [==============================] - 0s 121us/sample - loss: 1.2837 - accuracy: 0.5870
Epoch 83/500
138/138 [==============================] - 0s 120us/sample - loss: 1.2668 - accuracy: 0.5870
Epoch 84/500
138/138 [==============================] - 0s 115us/sample - loss: 1.2551 - accuracy: 0.5870
Epoch 85/500
138/138 [==============================] - 0s 120us/sample - loss: 1.2400 - accuracy: 0.6014
Epoch 86/500
138/138 [==============================] - 0s 118us/sample - loss: 1.2243 - accuracy: 0.6014
Epoch 87/500
138/138 [==============================] - 0s 108

Epoch 155/500
138/138 [==============================] - 0s 100us/sample - loss: 0.4157 - accuracy: 0.8913
Epoch 156/500
138/138 [==============================] - 0s 110us/sample - loss: 0.4060 - accuracy: 0.8841
Epoch 157/500
138/138 [==============================] - 0s 116us/sample - loss: 0.3996 - accuracy: 0.8913
Epoch 158/500
138/138 [==============================] - 0s 109us/sample - loss: 0.3912 - accuracy: 0.9058
Epoch 159/500
138/138 [==============================] - 0s 112us/sample - loss: 0.3829 - accuracy: 0.9058
Epoch 160/500
138/138 [==============================] - 0s 114us/sample - loss: 0.3768 - accuracy: 0.9203
Epoch 161/500
138/138 [==============================] - 0s 107us/sample - loss: 0.3694 - accuracy: 0.9348
Epoch 162/500
138/138 [==============================] - 0s 116us/sample - loss: 0.3608 - accuracy: 0.9275
Epoch 163/500
138/138 [==============================] - 0s 117us/sample - loss: 0.3539 - accuracy: 0.9275
Epoch 164/500
138/138 [==============

138/138 [==============================] - 0s 101us/sample - loss: 0.0897 - accuracy: 0.9928
Epoch 232/500
138/138 [==============================] - 0s 93us/sample - loss: 0.0883 - accuracy: 0.9928
Epoch 233/500
138/138 [==============================] - 0s 79us/sample - loss: 0.0870 - accuracy: 0.9928
Epoch 234/500
138/138 [==============================] - 0s 107us/sample - loss: 0.0849 - accuracy: 0.9928
Epoch 235/500
138/138 [==============================] - 0s 98us/sample - loss: 0.0838 - accuracy: 0.9928
Epoch 236/500
138/138 [==============================] - 0s 112us/sample - loss: 0.0820 - accuracy: 0.9928
Epoch 237/500
138/138 [==============================] - 0s 98us/sample - loss: 0.0811 - accuracy: 0.9928
Epoch 238/500
138/138 [==============================] - 0s 117us/sample - loss: 0.0794 - accuracy: 0.9928
Epoch 239/500
138/138 [==============================] - 0s 117us/sample - loss: 0.0776 - accuracy: 0.9928
Epoch 240/500
138/138 [==============================] 

138/138 [==============================] - 0s 103us/sample - loss: 0.0268 - accuracy: 1.0000
Epoch 308/500
138/138 [==============================] - 0s 118us/sample - loss: 0.0265 - accuracy: 1.0000
Epoch 309/500
138/138 [==============================] - 0s 104us/sample - loss: 0.0263 - accuracy: 1.0000
Epoch 310/500
138/138 [==============================] - 0s 117us/sample - loss: 0.0258 - accuracy: 1.0000
Epoch 311/500
138/138 [==============================] - 0s 125us/sample - loss: 0.0253 - accuracy: 1.0000
Epoch 312/500
138/138 [==============================] - 0s 127us/sample - loss: 0.0250 - accuracy: 1.0000
Epoch 313/500
138/138 [==============================] - 0s 125us/sample - loss: 0.0246 - accuracy: 1.0000
Epoch 314/500
138/138 [==============================] - 0s 126us/sample - loss: 0.0243 - accuracy: 1.0000
Epoch 315/500
138/138 [==============================] - 0s 124us/sample - loss: 0.0240 - accuracy: 1.0000
Epoch 316/500
138/138 [============================

138/138 [==============================] - 0s 122us/sample - loss: 0.0111 - accuracy: 1.0000
Epoch 384/500
138/138 [==============================] - 0s 124us/sample - loss: 0.0109 - accuracy: 1.0000
Epoch 385/500
138/138 [==============================] - 0s 154us/sample - loss: 0.0109 - accuracy: 1.0000
Epoch 386/500
138/138 [==============================] - 0s 150us/sample - loss: 0.0107 - accuracy: 1.0000
Epoch 387/500
138/138 [==============================] - 0s 123us/sample - loss: 0.0106 - accuracy: 1.0000
Epoch 388/500
138/138 [==============================] - 0s 119us/sample - loss: 0.0106 - accuracy: 1.0000
Epoch 389/500
138/138 [==============================] - 0s 132us/sample - loss: 0.0104 - accuracy: 1.0000
Epoch 390/500
138/138 [==============================] - 0s 120us/sample - loss: 0.0103 - accuracy: 1.0000
Epoch 391/500
138/138 [==============================] - 0s 123us/sample - loss: 0.0102 - accuracy: 1.0000
Epoch 392/500
138/138 [============================

138/138 [==============================] - 0s 124us/sample - loss: 0.0057 - accuracy: 1.0000
Epoch 460/500
138/138 [==============================] - 0s 126us/sample - loss: 0.0056 - accuracy: 1.0000
Epoch 461/500
138/138 [==============================] - 0s 122us/sample - loss: 0.0056 - accuracy: 1.0000
Epoch 462/500
138/138 [==============================] - 0s 129us/sample - loss: 0.0055 - accuracy: 1.0000
Epoch 463/500
138/138 [==============================] - 0s 126us/sample - loss: 0.0055 - accuracy: 1.0000
Epoch 464/500
138/138 [==============================] - 0s 115us/sample - loss: 0.0055 - accuracy: 1.0000
Epoch 465/500
138/138 [==============================] - 0s 126us/sample - loss: 0.0054 - accuracy: 1.0000
Epoch 466/500
138/138 [==============================] - 0s 114us/sample - loss: 0.0054 - accuracy: 1.0000
Epoch 467/500
138/138 [==============================] - 0s 126us/sample - loss: 0.0053 - accuracy: 1.0000
Epoch 468/500
138/138 [============================

In [8]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-01-14 20:50:25.037947: W tensorflow/python/util/util.cc:319] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: chat_model/assets


In [9]:
#!pip install colorama

In [10]:

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("GL Bot.json") as file:
    data = json.load(file)


def chatbot():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)


Start messaging with the bot (type quit to stop)!


In [ ]:
chatbot()

User: hi
ChatBot: Hello! how can i help you ?
User: who are you
ChatBot: I am your virtual learning assistant
User: i want to enquire about machine learning
ChatBot: Link: Machine Learning wiki 
User: what are the working hours
ChatBot: I am your virtual learning assistant
User: 